In [3]:
!pip install torch-scatter
!pip install torch-sparse
!pip install torch-cluster
!pip install torch-spline-conv 
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 15.5 MB/s 
  Created wheel for torch-scatter: filename=torch_scatter-2.1.0-cp37-cp37m-linux_x86_64.whl size=3062821 sha256=f877f2f3f9adafb8a86d3090b5b6c647ac712d2b40462a05d2d2057c591a5502
  Stored in directory: /root/.cache/pip/wheels/2d/d1/15/8a2f0086896d156654a843fff4bdbeaf621cdd10310a0daad2
Successfully built torch-scatter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 15.6 MB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.15-cp37-cp37m-linux_x86_64.whl size=1577344 sha256=70041be1ddf651cafaf72b928486b761aebb3f912cc0359223014f48de0ab759
  Stored in directory: /root/.cache/pip/wheels/15/68/4d/1414be5c2c622bad35364e13213180797717b6d4b8923936dc
Successfully built torch-sparse
Looking in indexes: https://pypi.org/simple, https://u

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [67]:
## reproducibility를 위해 random seed 고정
import torch
import random
import torch.backends.cudnn as cudnn
import numpy as np

torch.manual_seed(256)
torch.cuda.manual_seed(256)
torch.cuda.manual_seed_all(256)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

In [68]:
import numpy as np
import pandas as pd
import pickle
import csv
import os
import torch
from torch_geometric.data import Data

In [69]:
from sklearn.preprocessing import LabelEncoder

item_encoder = LabelEncoder()

In [70]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/20221130-train.csv')

In [71]:
df.rename(columns={"Unnamed: 0": "session_id"}, inplace=True)

In [72]:
df

,session_id,0,1,2,3,4,5,6,7,8,...,504,505,506,507,508,509,510,511,label,sum
0,0,-0.18210,0.07730,-0.26900,-0.252200,0.340800,-0.20650,-0.02248,0.4219,-0.1059,...,0.543000,-0.12300,0.27980,0.11804,0.33960,1.1040,-0.18080,-0.27400,6,0
1,1,0.04443,0.05460,-0.12366,-0.042100,0.639600,0.25270,0.12620,0.4607,-0.0915,...,-0.056640,-0.04352,0.26500,0.19560,0.00894,0.9510,0.10187,0.13780,9,15
2,2,0.31150,-0.06003,-0.32640,0.091550,0.375700,-0.09204,0.25630,0.5290,-0.1687,...,0.006985,-0.36400,0.16750,0.40670,0.06335,0.5947,-0.13380,0.06290,9,-6
3,3,0.07544,-0.19980,-0.11127,0.023000,0.520500,0.01353,0.33300,0.6445,-0.1850,...,0.267600,-0.14090,0.05533,0.22730,0.21630,0.5220,0.08260,0.07886,4,2
4,4,0.14430,0.27370,-0.01513,0.013245,-0.019620,0.05264,0.36770,0.2988,0.0636,...,0.181200,-0.33300,0.10360,0.12030,0.17270,0.8086,-0.00516,0.48240,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,0.38450,0.00178,0.01033,-0.373500,0.003527,-0.17380,0.26510,0.8096,-0.3108,...,0.914600,-0.24740,0.10260,0.45400,0.04968,0.5880,0.34350,-0.19380,2,-15
49996,49996,0.32670,-0.05780,-0.43380,0.026610,0.405300,-0.30710,0.23140,0.4968,-0.1864,...,0.181400,-0.29250,0.11650,0.26640,-0.10910,1.0980,0.28900,0.18210,6,-10
49997,49997,0.24390,0.03134,-0.21250,-0.059170,0.381000,0.45310,0.38720,0.7183,-0.1107,...,0.281500,0.34080,0.27170,0.13350,-0.21750,0.5327,-0.01223,-0.01997,9,-5
49998,49998,0.31250,0.22380,-0.12320,-0.211800,-0.057160,0.06616,0.34810,0.2086,-0.1876,...,0.529000,0.02000,0.25950,0.08936,-0.13400,1.2440,0.29860,0.02827,1,0


In [73]:
df.rename(columns={"sum": "item_id"}, inplace=True)

In [74]:
df

,session_id,0,1,2,3,4,5,6,7,8,...,504,505,506,507,508,509,510,511,label,item_id
0,0,-0.18210,0.07730,-0.26900,-0.252200,0.340800,-0.20650,-0.02248,0.4219,-0.1059,...,0.543000,-0.12300,0.27980,0.11804,0.33960,1.1040,-0.18080,-0.27400,6,0
1,1,0.04443,0.05460,-0.12366,-0.042100,0.639600,0.25270,0.12620,0.4607,-0.0915,...,-0.056640,-0.04352,0.26500,0.19560,0.00894,0.9510,0.10187,0.13780,9,15
2,2,0.31150,-0.06003,-0.32640,0.091550,0.375700,-0.09204,0.25630,0.5290,-0.1687,...,0.006985,-0.36400,0.16750,0.40670,0.06335,0.5947,-0.13380,0.06290,9,-6
3,3,0.07544,-0.19980,-0.11127,0.023000,0.520500,0.01353,0.33300,0.6445,-0.1850,...,0.267600,-0.14090,0.05533,0.22730,0.21630,0.5220,0.08260,0.07886,4,2
4,4,0.14430,0.27370,-0.01513,0.013245,-0.019620,0.05264,0.36770,0.2988,0.0636,...,0.181200,-0.33300,0.10360,0.12030,0.17270,0.8086,-0.00516,0.48240,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,0.38450,0.00178,0.01033,-0.373500,0.003527,-0.17380,0.26510,0.8096,-0.3108,...,0.914600,-0.24740,0.10260,0.45400,0.04968,0.5880,0.34350,-0.19380,2,-15
49996,49996,0.32670,-0.05780,-0.43380,0.026610,0.405300,-0.30710,0.23140,0.4968,-0.1864,...,0.181400,-0.29250,0.11650,0.26640,-0.10910,1.0980,0.28900,0.18210,6,-10
49997,49997,0.24390,0.03134,-0.21250,-0.059170,0.381000,0.45310,0.38720,0.7183,-0.1107,...,0.281500,0.34080,0.27170,0.13350,-0.21750,0.5327,-0.01223,-0.01997,9,-5
49998,49998,0.31250,0.22380,-0.12320,-0.211800,-0.057160,0.06616,0.34810,0.2086,-0.1876,...,0.529000,0.02000,0.25950,0.08936,-0.13400,1.2440,0.29860,0.02827,1,0


In [75]:
temp = []
for i in range(512):
  temp.append(str(i))

In [76]:
tgt = ['sess_item_id','item_id'] + temp

In [77]:
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm

class YooChooseBinaryDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseBinaryDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return ['/content/drive/MyDrive/20201130-graph2.dataset']

    def download(self):
        pass
    
    def process(self):
        
        data_list = []

        # process by session_id
        grouped = df.groupby('session_id')
        for session_id, group in tqdm(grouped):
            
            sess_item_id = LabelEncoder().fit_transform(group.item_id)
            group = group.reset_index(drop=True)
            group['sess_item_id'] = sess_item_id
            print(group['sess_item_id'])
            
            node_features = group.loc[group.session_id==session_id, tgt].sort_values('sess_item_id').item_id.drop_duplicates().values

            node_features = torch.LongTensor(node_features).unsqueeze(1)
            target_nodes = group.sess_item_id.values[1:]
            source_nodes = group.sess_item_id.values[:-1]

            edge_index = torch.tensor([source_nodes,
                                   target_nodes], dtype=torch.long)
            x = node_features

            y = torch.FloatTensor([group.label.values[0]])

            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [78]:
train_dataset = YooChooseBinaryDataset(root='../')

In [79]:
train_dataset

YooChooseBinaryDataset(50000)

In [80]:
len(train_dataset)

50000

In [81]:
import pandas as pd
df1 = pd.read_csv('/content/drive/MyDrive/20221130-test.csv')
df1.rename(columns={"Unnamed: 0": "session_id"}, inplace=True)
df1.rename(columns={"sum": "item_id"}, inplace=True)
df1

,session_id,0,1,2,3,4,5,6,7,8,...,504,505,506,507,508,509,510,511,label,item_id
0,0,0.234400,0.049840,-0.44800,-0.3662,0.07697,-0.220000,0.08720,0.9460,-0.4084,...,0.5215,0.003767,0.35550,0.11523,-0.122860,0.5347,0.03108,0.2323,3,-3
1,1,0.481200,-0.031560,-0.06824,0.0570,0.15140,0.004078,0.48970,0.4920,0.1619,...,0.0992,-0.289000,0.00778,0.18040,0.270300,0.5490,-0.03265,-0.2542,8,3
2,2,0.143900,-0.001246,-0.24490,-0.1326,0.13770,0.089900,0.23940,0.7540,0.2712,...,0.6770,0.124760,-0.08344,0.39100,0.112600,0.6140,0.01251,0.0507,8,0
3,3,0.363300,-0.215200,-0.08685,-0.5660,0.64400,-0.493200,0.09045,0.1783,-0.3460,...,0.7515,-0.644000,0.09680,0.24870,0.049000,0.7320,-0.44600,-0.1598,0,-9
4,4,-0.024630,-0.201400,0.00935,-0.2213,-0.30440,0.156500,0.04504,0.2732,0.5586,...,0.3984,-0.527300,0.24740,-0.03687,0.348600,0.6284,0.01686,-0.2502,6,-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.261000,0.012500,-0.25660,-0.4375,0.38920,-0.120600,0.32520,1.0380,-0.1399,...,0.3900,-0.068400,0.32000,0.11163,0.011375,0.7810,-0.03190,0.2703,8,1
9996,9996,0.180500,0.098500,-0.24070,0.1516,0.09393,-0.528000,0.16800,0.5410,0.0872,...,0.4600,0.112550,0.02051,0.36230,-0.195800,0.9670,-0.01642,0.0477,3,-1
9997,9997,0.005203,-0.169300,-0.41240,0.2202,0.46800,-0.075740,0.37000,0.5780,0.4443,...,0.1343,0.187600,-0.13500,0.43950,-0.377200,0.7744,0.23630,0.0642,5,0
9998,9998,0.501000,0.154500,-0.02998,-0.3838,0.08120,0.244500,0.39500,0.7610,-0.1854,...,0.3372,-0.393800,0.09863,-0.05440,0.253000,0.3909,-0.01448,-0.0163,1,-5


In [82]:
temp = []
for i in range(512):
  temp.append(str(i))

In [83]:
tgt = ['sess_item_id','item_id'] + temp

In [84]:
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm

class YooChooseBinaryDataset2(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseBinaryDataset2, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return ['/content/drive/MyDrive/20201130-graph3.dataset']

    def download(self):
        pass
    
    def process(self):
        
        data_list = []

        # process by session_id
        grouped = df1.groupby('session_id')
        for session_id, group in tqdm(grouped):
            
            sess_item_id = LabelEncoder().fit_transform(group.item_id)
            group = group.reset_index(drop=True)
            group['sess_item_id'] = sess_item_id
            print(group['sess_item_id'])
            
            node_features = group.loc[group.session_id==session_id, tgt].sort_values('sess_item_id').item_id.drop_duplicates().values

            node_features = torch.LongTensor(node_features).unsqueeze(1)
            target_nodes = group.sess_item_id.values[1:]
            source_nodes = group.sess_item_id.values[:-1]

            edge_index = torch.tensor([source_nodes,
                                   target_nodes], dtype=torch.long)
            x = node_features

            y = torch.FloatTensor([group.label.values[0]])

            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [85]:
test_dataset = YooChooseBinaryDataset2(root='../')

In [86]:
len(train_dataset), len(test_dataset)

(50000, 10000)

In [95]:
from torch_geometric.data import DataLoader
batch_size= 8
train_loader = DataLoader(train_dataset, batch_size=batch_size)
# val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [96]:
# num_items = df.item_id.max() +1 
# num_items 

In [97]:
import torch
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops
class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='max') #  "Max" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]

        x_j = self.lin(x_j)
        x_j = self.act(x_j)
        
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]


        new_embedding = torch.cat([aggr_out, x], dim=1)
        
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)
        
        return new_embedding

In [98]:
embed_dim = 128
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = SAGEConv(embed_dim, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.item_embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() +1, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 10)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()  

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(x)
        x = self.item_embedding(x)
        x = x.squeeze(1)     
        
        x = F.relu(self.conv1(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)

        x = torch.nn.functional.softmax(self.lin3(x)).squeeze(1)

        return x

In [99]:
device = torch.device('cuda')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.CrossEntropyLoss()

In [100]:
def train():
    model.train()

    loss_all = 0
    for data in train_loader:

        data.to('cuda:0')
        
        optimizer.zero_grad()
        output = model(data)
        
        label = data.y.to(device)
        label = label.type(torch.LongTensor)
        label = label.to(device)
        
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)

In [101]:
from sklearn.metrics import roc_auc_score
def evaluate(loader):
    model.eval()

    predictions = []
    labels = []
    correct = 0
    correct_list = []

    with torch.no_grad():
        for data in loader:

            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

            predicted = np.argmax(pred)
            correct += (predicted == label)
            correct_list.append(100 * (correct / len(loader.dataset)))


    predictions = np.hstack(predictions)
    labels = np.hstack(labels)
    
    return np.mean(correct_list)

In [ ]:
test_acc_list = []
for epoch in range(10):
    loss = train()
    train_acc = evaluate(train_loader)
    test_acc = evaluate(test_loader)
    print(train_acc)
    print(test_acc)

    test_acc_list.append(test_acc)

In [ ]:
np.mean(test_acc_list)